# OMERO Micro-SAM Workflow




## 1. Setup and Installation

In [1]:
# Import the streamlined package
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
)

# System imports
import os
from pathlib import Path

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
# Check dependencies
try:
    import ezomero
    print(f"🔗 OMERO functionality: ✅ Available")
except ImportError:
    print(f"🔗 OMERO functionality: ❌ Install with: pip install -e .[omero]")

try:
    import keyring
    print(f"🔑 Keyring support: ✅ Available")
except ImportError:
    print(f"🔑 Keyring support: ⚠️ Not available (manual password entry only)")

📦 omero-annotate-ai version: 0.1.0
🔗 OMERO functionality: ✅ Available
🔑 Keyring support: ✅ Available


## 2. OMERO Connection

In [2]:
# Create and display the OMERO connection widget
print("🔌 OMERO Connection Setup")
print("Use the widget below to connect to your OMERO server:")
print("  • Fill in server details (host, username, password)")
print("  • Test connection before proceeding")
print("  • Choose password storage duration if desired")
print("  • Click 'Save & Connect' to establish connection")
print()

conn_widget = create_omero_connection_widget()
conn_widget.display()

print("\n📝 Next Step: Run the cell below after connecting to OMERO")

🔌 OMERO Connection Setup
Use the widget below to connect to your OMERO server:
  • Fill in server details (host, username, password)
  • Test connection before proceeding
  • Choose password storage duration if desired
  • Click 'Save & Connect' to establish connection

📄 Loaded configuration from connection history: root@localhost (system)
🔐 Password loaded from keychain (no expiration)



📝 Next Step: Run the cell below after connecting to OMERO


In [3]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("❌ No OMERO connection established. Please use the widget above to connect.")

print("✅ OMERO connection established!")
print(f"👤 User: {conn.getUser().getName()}")
print(f"🏢 Group: {conn.getGroupFromContext().getName()}")
print(f"🔐 Secure: {conn.isSecure()}")
print(f"🔗 Connection ready for workflow setup")

✅ OMERO connection established!
👤 User: root
🏢 Group: system
🔐 Secure: True
🔗 Connection ready for workflow setup


## 3. Workflow Configuration

In [4]:
# Create and display the workflow widget
print("🔬 OMERO Annotation Workflow Setup")
print("Follow the sequential workflow below:")
print("  1. Select working directory")
print("  2. Choose OMERO container")
print("  3. Check existing annotation tables")
print("  4. Configure micro-SAM parameters")
print("  5. Save configuration")
print()

workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

print("\n📝 Next Step: Complete the workflow above, then run the cell below")

🔬 OMERO Annotation Workflow Setup
Follow the sequential workflow below:
  1. Select working directory
  2. Choose OMERO container
  3. Check existing annotation tables
  4. Configure micro-SAM parameters
  5. Save configuration




📝 Next Step: Complete the workflow above, then run the cell below


## 4. Pipeline Execution


In [5]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

# Display configuration summary
print("\n📋 Configuration Summary:")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🎯 Training Set: {config.name}")
print(f"   🔬 Model: {config.ai_model.model_type}")
print(f"   📺 Channel: {config.spatial_coverage.channels}")
print(f"   📁 Output: {config.output.output_directory}")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"   🧩 Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})"
    )

if config.spatial_coverage.three_d:
    print(f"   🧊 3D processing: Enabled")


print(f"\n📊 Processing: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")


📋 Configuration Summary:
   📦 Container: plate (ID: 2)
   🎯 Training Set: micro_sam_incucyte_20250909_204150
   🔬 Model: vit_b_lm
   📺 Channel: [0]
   📁 Output: D:\Maarten\Data\zerocost
   🔄 Resume from Table: False
   📖 Read-only Mode: False
   🧩 Patches: 1 per image (512×512)

📊 Processing: 3 training + 3 validation


In [ ]:
from omero_annotate_ai.core.annotation_config import load_config_from_yaml

config = load_config_from_yaml(
    "D:\Maarten\Data\incucyte_training\\annotation_config.yaml"
)


In [6]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"🔍 Validating {container_type} with ID {container_id}...")

# Validate container exists
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")

print(f"✅ Found {container_type}: {container.getName()}")
if container.getDescription():
    print(f"📝 Description: {container.getDescription()}")

🔍 Validating plate with ID 2...
✅ Found plate: incucyte


In [7]:
# Run the complete annotation pipeline
print("🚀 Starting annotation pipeline...")
print(f"   🔬 Using micro-SAM model: {config.ai_model.model_type}")

if config.processing.batch_size == 0:
    print(f"   ⚡ Processing: All images in one batch")
else:
    print(f"   📦 Processing: Batches of {config.processing.batch_size} images")

print(f"   🎨 Napari will open for interactive annotation")
print(f"   📝 Close napari windows when annotation is complete")

try:
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n🎉 Annotation pipeline completed successfully!")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.output.output_directory}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()
    raise

🚀 Starting annotation pipeline...
   🔬 Using micro-SAM model: vit_b_lm
   ⚡ Processing: All images in one batch
   🎨 Napari will open for interactive annotation
   📝 Close napari windows when annotation is complete
Loading images from plate 2
Found 88 images
Processing 88 images with model: vit_b_lm
Preparing processing units...
Creating initial OMERO table...
📋 Created/replaced tracking table 'micro_sam_training_micro_sam_incucyte_20250909_204150' with 6 units
   Container: plate 2
   Table ID: 39
Created OMERO table with ID: 39
Found 6 unprocessed annotations

🔄 Processing batch 1/1
   📦 Batch size: 6
   📊 Config state before: 0/6 processed
   🎨 Running micro-SAM annotation...
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
✅ Successfully loaded 1 images
Loaded 2D data shape: (1040, 1408)
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
✅ Successfully loaded 1 images
Loaded 2D data shape:

Precompute state for files: 100%|███████████████████████████████| 6/6 [00:01<00:00,  3.52it/s]


Precomputation took 2.414928436279297 seconds (= 00:02 minutes)
The first image to annotate is image number 0


   ✅ micro-SAM completed, got results: <class 'dict'>
   🔄 Processing annotation results...
� Processing annotation results...
Found 0 annotation files for 6 metadata entries
⚠️ No annotation file for metadata entry 1
⚠️ No annotation file for metadata entry 2
⚠️ No annotation file for metadata entry 3
⚠️ No annotation file for metadata entry 4
⚠️ No annotation file for metadata entry 5
⚠️ No annotation file for metadata entry 6
📊 Updated 0/6 annotations
📈 Config now has 0/6 processed annotations
   📊 Config state after: 0/6 processed
   📋 Updating OMERO table...
📋 Updating OMERO table...
🗑️ Deleting table: micro_sam_training_micro_sam_incucyte_20250909_204150
🗑️ Deleting table: micro_sam_training_micro_sam_incucyte_20250909_204150
✅ Successfully deleted table 39
📋 Created/replaced tracking table 'micro_sam_training_micro_sam_incucyte_20250909_204150' with 6 units
   Container: plate 2
   Table ID: 40
OMERO table updated successfully (ID: 40)
   ✅ OMERO table updated (ID: 40)
   💾 Auto

Traceback (most recent call last):
  File "C:\Users\maart\AppData\Local\Temp\ipykernel_19032\1781668113.py", line 14, in <module>
    table_id, processed_images = pipeline.run_full_workflow()
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Maarten\Code\omero_annotate_ai\src\omero_annotate_ai\core\annotation_pipeline.py", line 945, in run_full_workflow
    self._upload_annotation_config_to_omero()
  File "D:\Maarten\Code\omero_annotate_ai\src\omero_annotate_ai\core\annotation_pipeline.py", line 351, in _upload_annotation_config_to_omero
    id = upload_annotation_config_to_omero(
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Maarten\Code\omero_annotate_ai\src\omero_annotate_ai\omero\omero_functions.py", line 147, in upload_annotation_config_to_omero
    id = ezomero.post_file_annotation(conn,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Maarten\Code\omero_annotate_ai\.pixi\envs\default\Lib\site-packages\ezomero\_ezomero.py", line 63, in wrapper

TypeError: file_path must be of type `str`

In [ ]:
# Export configuration for future use
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"💾 Configuration saved to: {config_path}")
except Exception as e:
    print(f"⚠️ Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"💾 Configuration saved to: {config_filename}")

print(f"\n📋 Configuration Summary:")
print(f"   Name: {config.name}")
print(f"   Description: {config.description}")
print(f"   Output Directory: {config.output.output_directory}")
print(f"   Model Type: {config.ai_model.model_type}")

print(f"\n🔄 To reuse this configuration:")
print(f"```python")
print(f"from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")
print(f"```")

## 6. Cleanup

Close the OMERO connection when finished.

In [ ]:
# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print(f"\n🎉 Streamlined workflow completed!")
print(f"📊 Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")
print(f"💾 Configuration saved for future use")
print(f"✨ Ready for next annotation workflow!")